### Day 2

> Descriptors

Adjusting attribute access.

* `__get__`, `__set__`, `__del__` 

In [2]:
class A:
    x = 10

In [3]:
a = A()

In [5]:
a.x

10

In [7]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'x': 10,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [8]:
a.__dict__

{}

In [9]:
class D:
    def __get__(self, obj, objtype=None):
        print("D.__get__", obj)
        return 10

In [10]:
class A:
    x = D()

In [11]:
a = A()

In [12]:
a.x

D.__get__ <__main__.A object at 0x7fc7c01828e0>


10

In [13]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'x': <__main__.D at 0x7fc7c0182a00>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [14]:
import logging
logging.basicConfig(level=logging.INFO)

In [15]:
class LoggedAgeAccess:
    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info("accessing %r giving %r", 'age', value)
        return value
        
    def __set__(self, obj, value):
        logging.info("updating %r to %r", 'age', value)
        obj._age = value

class Person:
    age = LoggedAgeAccess()
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def birthday(self):
        self.age += 1

In [16]:
mary = Person("mary", 35)
dave = Person("dave", 32)

INFO:root:updating 'age' to 35
INFO:root:updating 'age' to 32


In [17]:
mary.__dict__

{'name': 'mary', '_age': 35}

In [18]:
mary.birthday()

INFO:root:accessing 'age' giving 35
INFO:root:updating 'age' to 36


In [26]:
class LoggedAccess:
    
    def __set_name__(self, owner, name): # pep 487
        print(owner, name)
        self.public_name = name
        self.private_name = "_" + name
    
    def __get__(self, obj, objtype=None):
        value = getattr(obj, self.private_name)
        logging.info("accessing %r giving %r", self.public_name, value)
        return value
        
    def __set__(self, obj, value):
        logging.info("updating %r to %r", self.public_name, value)
        setattr(obj, self.private_name, value)

class Person:
    age = LoggedAccess()
    name = LoggedAccess()
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def birthday(self):
        self.age += 1

<class '__main__.Person'> age
<class '__main__.Person'> name


In [27]:
mary = Person("mary", 25)
dave = Person("dave", 30)

INFO:root:updating 'name' to 'mary'
INFO:root:updating 'age' to 25
INFO:root:updating 'name' to 'dave'
INFO:root:updating 'age' to 30


In [28]:
vars(mary)

{'_name': 'mary', '_age': 25}

In [29]:
mary.__dict__

{'_name': 'mary', '_age': 25}

In [30]:
# class A:
#    x = CachedAttr("x") 

In [33]:
class LoggedAccess:
    
    def __set_name__(self, owner, name): # pep 487
        print(owner, name)
        self.public_name = name
        self.private_name = "_" + name
    
    def __get__(self, obj, objtype=None):
        value = getattr(obj, self.private_name)
        logging.info("accessing %r giving %r", self.public_name, value)
        return value
        
    def __set__(self, obj, value):
        logging.info("updating %r to %r", self.public_name, value)
        setattr(obj, self.private_name, value)


**Task**: Descriptors, implementation

* implement `IntRange` which would allow to verify constraints on an attribute
* two possible constraints are "min" and "max" values for the integer value

Any operation that would set the attribute to an invalid value should result in some exception (e.g. ValueError).

In [32]:
class Record:
    x = IntRange(min=10, max=20)
    y = IntRange(min=0)
    
record = Record()
record.x = 10 # ok
record.y = -1 # fail with ValueError

NameError: name 'IntRange' is not defined

In [34]:
class IntRange:
    
    def __init__(self, min=None, max=None):
        self.min = min
        self.max = max
    
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = "_" + name
    
    def __get__(self, obj, objectype):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        if self.min is not None:
            if value < self.min:
                raise ValueError("value too small")
        if self.max is not None:
            if value > self.max:
                raise ValueError("value too large")
        setattr(obj, self.private_name, value)

In [35]:
class Record:
    x = IntRange(min=10, max=20)
    y = IntRange(min=0)

In [36]:
record = Record()

In [37]:
record.x = 10

In [42]:
record.x = 100

ValueError: value too large

In [43]:
from abc import ABC, abstractmethod

class Validator(ABC):
    
    def __set_name__(self, owner, name):
        self.private_name = "_" + name
        
    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)
    
    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)
        
    @abstractmethod
    def validate(self, value):
        pass
    

In [47]:
class OneOf(Validator):
    def __init__(self, *options):
        self.options = set(options)
    
    def validate(self, value):
        if value not in self.options:
            raise ValueError(f"expected {value} to be one of {self.options}")

In [50]:
class Number(Validator):
    def __init__(self, min=None, max=None):
        self.min = min
        self.max = max
    
    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("number required")
        if self.min is not None:
            if value < self.min:
                raise ValueError("value too small")
        if self.max is not None:
            if value > self.max:
                raise ValueError("value too large")

In [53]:
class String(Validator):
    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate # function
    
    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError("string required")
        if self.minsize is not None and len(value) < self.minsize:
            raise ValueError("too short")
        if self.maxsize is not None and len(value) > self.maxsize:
            raise ValueError("too long")
        if self.predicate is not None and not self.predicate(value):
            raise ValueError("predicate needs to evaluate to true")
        

In [54]:
class Component:
    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf("wood", "metal", "plastic")
    quantity = Number(min=0)
    
    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity

In [55]:
c = Component("PART123", "wood", 10)

In [58]:
d = Component("ABC", "glas", 10)

ValueError: expected glas to be one of {'plastic', 'metal', 'wood'}

Metaclasses

* class of a class

Class definition

* MRO entrie are resolved
* find the metaclass
* class namespace is prepared
* class body is executed
* the class object is created

Use cases:
    
* avoid repetition with class decorators
* validation of subclasses
* registering subclasses
* declarative GUI creation
* django ORM (adding attributes)

In [61]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    pass

In [62]:
type(MySubclass)

__main__.Meta

In [66]:
C = type("C", (), {"x": 1})

In [67]:
type(C)

type

In [68]:
C.x

1

In [71]:
for i in range(10):
    class A:
        pass

In [74]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        k = super().__new__(cls, name, bases, dct)
        k.abc = 123
        return k

In [75]:
class A(metaclass=Meta):
    pass

In [76]:
type(A)

__main__.Meta

In [77]:
A.abc

123

In [80]:
class MyMeta(type):
    def __new__(cls, name, bases, dct):
        print("MyMeta.__new__")
        inst = super().__new__(cls, name, bases, dct)
        return inst

In [81]:
class A(metaclass=MyMeta):
    
    def __new__(cls, *arg, **kwargs):
        print("A.__new__")
        return super().__new__(cls, *args, **kwargs)
    
    def __init__(self):
        print("A.__init__")

MyMeta.__new__


In [82]:
class DocsRequired(type):
    def __new__(cls, name, bases, dct):
        inst = super().__new__(cls, name, bases, dct)
        if not inst.__doc__:
            raise LookupError("docs missing")
        return inst

In [83]:
class A(metaclass=DocsRequired):
    pass

LookupError: docs missing

In [84]:
class A(metaclass=DocsRequired):
    """
    Ok.
    """

In [91]:
class Final(type):
    def __init__(cls, name, bases, dct):
        for b in bases:
            if isinstance(b, Final):
                raise RuntimeError("cannot extend class")

In [92]:
class A(metaclass=Final):
    pass

In [93]:
class B(A):
    pass

RuntimeError: cannot extend class

## Iterators

* iterable -> `__iter__`
* iterator -> `__next__`

* builtin function: `iter` 

In [95]:
s = [1, 2, 3]

In [97]:
i = iter(s)

In [98]:
i

In [100]:
s.__iter__()

In [101]:
i

In [102]:
i.__next__

<method-wrapper '__next__' of list_iterator object at 0x7fc79f39b310>

In [103]:
while True:
    try:
        print(i.__next__())
    except StopIteration:
        break

1
2
3


In [104]:
for c in "abc":
    print(c)

a
b
c


In [105]:
!ls

Scratch1.ipynb	Scratch.ipynb  ScratchLimited.ipynb


In [107]:
with open("Scratch.ipynb") as f:
    for line in f:
        pass

In [117]:
class A:
    
    def __init__(self):
        self.i = 0
        
    def __iter__(self):
        self.i = 0
        return self
    
    def __next__(self):
        self.i += 1
        if self.i > 3:
            raise StopIteration
        return self.i

In [125]:
a = A()

In [126]:
a

In [127]:
for i in a:
    print(i)

1
2
3


In [128]:
i = iter(a)

In [129]:
while True:
    try:
        print(i.__next__())
    except StopIteration:
        break

1
2
3


infinite iterator

In [130]:
import random

In [131]:
class RandomNumbers:
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return random.randint(0, 100)

In [132]:
rn = RandomNumbers()

In [134]:
# for i in rn:
#     print(i)

In [135]:
import itertools

In [137]:
list(itertools.islice(rn, 20, 22))

[79, 31]

**1** "Todolist"

Implement a basic TODO-List class

* implement a separate Item class
* each Item has a name and done attribute
* a list of Items should be sorted so that items that are "done" come last

Example iteraction:

```python
t = TodoList()
t.items.append(Item("send notification", done=True))
t.items.append(Item("test sirens"))
t.items.append(Item("test sms"))

print(t)

for task in sorted(t):
    print(task)

# <TodoList with 3 items>
# Item test sirens done=False
# Item test sms done=False
# Item send notification done=True
```

In [139]:
class Item:
    
    def __str__(self):
        pass

class TodoList:
    
    def __str__(self):
        pass